In [1]:
import numpy as np
import scipy as sc
import SALib as salib
from SALib.sample import sobol as salib_sample_sobol
from SALib.analyze import delta, dgsm, fast, ff, hdmr, morris, pawn, rbd_fast, sobol

from dwave.system.samplers import DWaveSampler

import matplotlib.pyplot as plt

In [2]:
with open('../API_Token.txt') as file:
    token = file.readline().rstrip()
sampler = DWaveSampler(token=token, architecture='pegasus', region='eu-central-1')

In [3]:
sampler.properties

{'num_qubits': 5760,
 'qubits': [30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  159,
  160,
  161,
  162,
  163,
  164,
  165,
  166,
  167,
  168,
  169,
  170,
  171,
  172,
  173,
  174,
  175,
  176,
  177,
  1

In [4]:
#num_qubits, coupler = len(couplers), h_range, j_range, anneal_offset_step,anneal_offset_step_phi0,annealing_time_range
#default_annealing_time, default_programming_thermalization, default_readout_thermalization, extended_j_range, h_gain_schedule_range
#max_anneal_schedule_points, max_h_gain_schedule_points, num_reads_range, per_qubit_coupling_range, problem_run_duration_range
#default_annealing_time, qpu_delay_time_per_sample, default_readout_thermalization, decorrelation_max_nominal_anneal_time, decorrelation_time_range
#programming_thermalization_range, readout_thermalization_range
print('num_qubits ', sampler.properties['num_qubits'])
print('num couplers ', len(sampler.properties['couplers']))
print('h_range ', sampler.properties['h_range'])
print('h_gain_schedule_range ', sampler.properties['h_gain_schedule_range'])
print('max_h_gain_schedule_points ', sampler.properties['max_h_gain_schedule_points'])
print('j_range ', sampler.properties['j_range'])
print('extended_j_range ', sampler.properties['extended_j_range'])
print('per_qubit_coupling_range ', sampler.properties['per_qubit_coupling_range'])
print('anneal_offset_step ', sampler.properties['anneal_offset_step'])
print('anneal_offset_step_phi0 ', sampler.properties['anneal_offset_step_phi0'])
print('max_anneal_schedule_points ', sampler.properties['max_anneal_schedule_points'])
print('annealing_time_range ', sampler.properties['annealing_time_range'])
print('default_annealing_time ', sampler.properties['default_annealing_time'])
print('programming_thermalization_range ', sampler.properties['programming_thermalization_range'])
print('default_programming_thermalization ', sampler.properties['default_programming_thermalization'])
print('readout_thermalization_range ', sampler.properties['readout_thermalization_range'])
print('default_readout_thermalization ', sampler.properties['default_readout_thermalization'])
print('num_reads_range ', sampler.properties['num_reads_range'])
print('problem_run_duration_range ', sampler.properties['problem_run_duration_range'])
print('qpu_delay_time_per_sample ', sampler.properties['problem_timing_data']['qpu_delay_time_per_sample'])
print('decorrelation_max_nominal_anneal_time ', sampler.properties['problem_timing_data']['decorrelation_max_nominal_anneal_time'])
print('decorrelation_time_range ', sampler.properties['problem_timing_data']['decorrelation_time_range'])

num_qubits  5760
num couplers  40050
h_range  [-4.0, 4.0]
h_gain_schedule_range  [-4.0, 4.0]
max_h_gain_schedule_points  20
j_range  [-1.0, 1.0]
extended_j_range  [-2.0, 1.0]
per_qubit_coupling_range  [-18.0, 15.0]
anneal_offset_step  -0.00013511442361210154
anneal_offset_step_phi0  1.277433705713884e-05
max_anneal_schedule_points  12
annealing_time_range  [0.5, 2000.0]
default_annealing_time  20.0
programming_thermalization_range  [0.0, 10000.0]
default_programming_thermalization  1000.0
readout_thermalization_range  [0.0, 10000.0]
default_readout_thermalization  0.0
num_reads_range  [1, 10000]
problem_run_duration_range  [0.0, 1000000.0]
qpu_delay_time_per_sample  21.02
decorrelation_max_nominal_anneal_time  2000.0
decorrelation_time_range  [500.0, 10000.0]


In [5]:
salib_problem = {
    'num_vars': 3,
    'names': ['annealing_time', 'programming_thermalization', 'readout_thermalization'],
    'bounds': [sampler.properties['annealing_time_range'],
               sampler.properties['programming_thermalization_range'],
               sampler.properties['readout_thermalization_range']]
}
salib_problem['bounds'] = 3*[[0,1]]
salib_problem

{'num_vars': 3,
 'names': ['annealing_time',
  'programming_thermalization',
  'readout_thermalization'],
 'bounds': [[0, 1], [0, 1], [0, 1]]}

In [6]:
N = 2**6
salib_sobol_samples = salib_sample_sobol.sample(salib_problem, N, calc_second_order=True, scramble=True)
print('N ', N, ', D ', salib_problem['num_vars'])
print('num samples: N(2D+2) = ', N*(2*salib_problem['num_vars']+2))
print('shape', salib_sobol_samples.shape)
print(salib_problem['names'])
salib_sobol_samples

N  64 , D  3
num samples: N(2D+2) =  512
shape (512, 3)
['annealing_time', 'programming_thermalization', 'readout_thermalization']


array([[0.90771037, 0.04167495, 0.29981064],
       [0.33097662, 0.04167495, 0.29981064],
       [0.90771037, 0.92783042, 0.29981064],
       ...,
       [0.03935677, 0.96628546, 0.68729409],
       [0.03935677, 0.32926554, 0.14097087],
       [0.03935677, 0.32926554, 0.68729409]])

In [7]:
m = int(np.log2(N*(2*salib_problem['num_vars']+2)))
scipy_sobol_sampler = sc.stats.qmc.Sobol(d=salib_problem['num_vars'], scramble=True, bits=None, seed=None, optimization=None)
scipy_sobol_samples = scipy_sobol_sampler.random_base2(m=m)
lb = [sublist[0] for sublist in salib_problem['bounds']]
ub = [sublist[1] for sublist in salib_problem['bounds']]
scipy_sobol_samples = sc.stats.qmc.scale(scipy_sobol_samples, l_bounds=lb, u_bounds=ub)
print('N ', N, ', D ', salib_problem['num_vars'])
print('num samples: N(2D+2) = ', N*(2*salib_problem['num_vars']+2))
print('shape', scipy_sobol_samples.shape)
print(salib_problem['names'])
scipy_sobol_samples

N  64 , D  3
num samples: N(2D+2) =  512
shape (512, 3)
['annealing_time', 'programming_thermalization', 'readout_thermalization']


array([[0.55235203, 0.8232461 , 0.81955841],
       [0.15073901, 0.00665726, 0.05954493],
       [0.3056033 , 0.7061817 , 0.52912463],
       ...,
       [0.30853387, 0.40359729, 0.98422063],
       [0.14976138, 0.82101812, 0.45213331],
       [0.55332795, 0.00528274, 0.67699816]])

In [8]:
discrepancy_methods = ['CD', 'WD', 'MD', 'L2-star']
for method in discrepancy_methods:
    salib_rescaled = sc.stats.qmc.scale(salib_sobol_samples, l_bounds=lb, u_bounds=ub, reverse=True)
    scipy_rescaled = sc.stats.qmc.scale(scipy_sobol_samples, l_bounds=lb, u_bounds=ub, reverse=True)
    salib_discrp = sc.stats.qmc.discrepancy(method=method, sample=salib_rescaled)
    scipy_discrp = sc.stats.qmc.discrepancy(method=method, sample=scipy_rescaled)
    print(method, ' salib: ', '{:.3e}'.format(salib_discrp))
    print(method, ' scipy: ', '{:.3e}'.format(scipy_discrp), '\n')

CD  salib:  2.056e-04
CD  scipy:  9.842e-06 

WD  salib:  5.286e-04
WD  scipy:  2.945e-05 

MD  salib:  5.173e-04
MD  scipy:  2.477e-05 

L2-star  salib:  7.425e-03
L2-star  scipy:  1.956e-03 



In [9]:
def test_function(input_1, input_2, input_3):
    one, two, three = 1, 2, 3
    oneone, twotwo, threethree = 1, 2, 3
    onetwo, onethree, twothree = 1, 2, 3
    output = one*input_1 + two*input_2 + three*input_3
    output += oneone*input_1*input_1 + twotwo*input_2*input_2 + threethree*input_3*input_3
    output += onetwo*input_1*input_2 + onethree*input_1*input_3 + twothree*input_2*input_3
    return output

In [10]:
salib_output = np.zeros(len(salib_sobol_samples))
scipy_output = np.zeros(len(salib_sobol_samples))
for i in range(len(salib_sobol_samples)):
    set_sa = salib_sobol_samples[i]
    set_sc = scipy_sobol_samples[i]
    salib_output[i] = test_function(set_sa[0], set_sa[1], set_sa[2])
    scipy_output[i] = test_function(set_sc[0], set_sc[1], set_sc[2])
print(salib_output)
print(scipy_output)

[ 3.6071582   1.94617466  8.69914317  5.81140601 11.67357743  3.81625047
  6.52708385  9.16734611 10.33000437 11.91093462  7.01627309  6.18912733
  3.50312318  7.4691935   8.42599602  4.61198201  2.10618774  2.21897082
  2.13348089  1.73802879  1.76392138  1.84364091  2.24662139  1.86989093
 11.29761957 12.17625826  8.54955437 13.78350841 10.80787601 14.74177009
  9.36196163 11.69990626  9.78597927 11.77453425 11.25247316  7.4770201
  8.81068373  9.27574527 13.32246667 10.69084743  5.80748531  6.06510989
  4.18168125  4.78981971  3.27349815  5.02316542  4.41392585  3.48146389
  4.41726051  5.37603985  3.69311296  1.32906545  0.84316084  1.99750969
  4.60502369  1.46473647  3.91608412  2.98952926  5.80233957 11.67012245
 14.383274   10.20360816  4.77267905 12.81365404  6.53829956  3.92055106
  5.62683478  8.84312193  7.80955544  5.84064151  3.14389402  4.94188276
  5.06970798  7.00107259  5.51335259  5.92762289  6.39478948  7.99130119
  7.48385304  8.49760362  6.4267534   7.39379104  9.

In [11]:
salib_output.size % (3)
salib_sobol_samples.shape

(512, 3)

In [14]:
# delta, dgsm, fast, ff, hdmr, morris, pawn, rbd_fast, sobol
analyses_salib = {}
analyses_scipy = {}
tmp_dict_salib = {'delta': {'problem':salib_problem, 'X':salib_sobol_samples, 'Y':salib_output, 'num_resamples': 100, 'conf_level': 0.95, 'print_to_console': False, 'seed': None}, 
            'dgsm': {'problem':salib_problem, 'X':salib_sobol_samples, 'Y':salib_output, 'num_resamples': 100, 'conf_level': 0.95, 'print_to_console': False, 'seed': None}, 
            #'fast': {'problem':salib_problem, 'Y':salib_output, 'M':4, 'num_resamples': 100, 'conf_level': 0.95, 'print_to_console': False, 'seed': None},
            #'ff': {'problem':salib_problem, 'X':salib_sobol_samples, 'Y':salib_output, 'second_order': True, 'print_to_console': False, 'seed': None}, 
            'hdmr': {'problem':salib_problem, 'X':salib_sobol_samples, 'Y':salib_output, 'maxorder': 2, 'maxiter': 100, 'm': 2, 'K':20, 'R':None, 'alpha':0.95, 'lambdax':0.01, 'print_to_console': False, 'seed': None},
            'morris': {'problem':salib_problem, 'X':salib_sobol_samples, 'Y':salib_output, 'num_resamples': 100, 'conf_level': 0.95, 'num_levels': 4, 'print_to_console': False, 'seed': None},
            'pawn': {'problem':salib_problem, 'X':salib_sobol_samples, 'Y':salib_output, 'S': 10, 'print_to_console': False, 'seed': None},
            'rbd_fast': {'problem':salib_problem, 'X':salib_sobol_samples, 'Y':salib_output, 'M':10, 'num_resamples': 100, 'conf_level': 0.95, 'print_to_console': False, 'seed': None},
            'sobol': {'problem':salib_problem, 'Y':salib_output, 'calc_second_order': True, 'num_resamples': 100, 'conf_level': 0.95, 'print_to_console': False, 'parallel': False, 'n_processors': None, 'keep_resamples': False, 'seed': None}
            }
tmp_dict_scipy = tmp_dict_salib.copy()
for key in analyses_salib.keys():
    tmp_dict_scipy[key].update({'X':scipy_sobol_samples})
    tmp_dict_scipy[key].update({'Y':scipy_output})
for method, params in tmp_dict_salib.items():
    print(method, ' ', salib.analyze.__dict__[method])
    analyses_salib[method] = salib.analyze.__dict__[method].analyze(**params)
for method, params in tmp_dict_scipy.items():
    print(method, ' ', salib.analyze.__dict__[method])
    analyses_scipy[method] = salib.analyze.__dict__[method].analyze(**params)

delta   <module 'SALib.analyze.delta' from 'c:\\ProgramData\\Anaconda3\\envs\\masterth_python_3_12\\Lib\\site-packages\\SALib\\analyze\\delta.py'>
dgsm   <module 'SALib.analyze.dgsm' from 'c:\\ProgramData\\Anaconda3\\envs\\masterth_python_3_12\\Lib\\site-packages\\SALib\\analyze\\dgsm.py'>
hdmr   <module 'SALib.analyze.hdmr' from 'c:\\ProgramData\\Anaconda3\\envs\\masterth_python_3_12\\Lib\\site-packages\\SALib\\analyze\\hdmr.py'>
morris   <module 'SALib.analyze.morris' from 'c:\\ProgramData\\Anaconda3\\envs\\masterth_python_3_12\\Lib\\site-packages\\SALib\\analyze\\morris.py'>
pawn   <module 'SALib.analyze.pawn' from 'c:\\ProgramData\\Anaconda3\\envs\\masterth_python_3_12\\Lib\\site-packages\\SALib\\analyze\\pawn.py'>
rbd_fast   <module 'SALib.analyze.rbd_fast' from 'c:\\ProgramData\\Anaconda3\\envs\\masterth_python_3_12\\Lib\\site-packages\\SALib\\analyze\\rbd_fast.py'>
sobol   <module 'SALib.analyze.sobol' from 'c:\\ProgramData\\Anaconda3\\envs\\masterth_python_3_12\\Lib\\site-packa

In [15]:
for name, analysis in analyses_salib.items():
    print(name)
    for key, value in analysis.items():
        print(' ', key, ' ', value, ' ', analyses_scipy[name][key])

delta
  delta   [0.08119659 0.18332502 0.32745711]   [0.08200687 0.18293209 0.32899981]
  delta_conf   [0.03087446 0.03205412 0.02799769]   [0.02418077 0.02947639 0.02710846]
  S1   [0.09806255 0.29389624 0.58307459]   [0.09806255 0.29389624 0.58307459]
  S1_conf   [0.05240619 0.05926112 0.05520244]   [0.0453268  0.06043998 0.0498614 ]
  names   ['annealing_time', 'programming_thermalization', 'readout_thermalization']   ['annealing_time', 'programming_thermalization', 'readout_thermalization']
dgsm
  vi   [537.87619721          inf          inf]   [537.87619721          inf          inf]
  vi_std   [3285.9564976          nan          nan]   [3285.9564976          nan          nan]
  dgsm   [5.36148879        inf        inf]   [5.36148879        inf        inf]
  dgsm_conf   [620.60613647          nan          nan]   [528.81454113          nan          nan]
  names   ['annealing_time', 'programming_thermalization', 'readout_thermalization']   ['annealing_time', 'programming_thermalizat